# *DATA SCIENCE / SECTION 1 / PROJECT*



# 🏆 프로젝트 목표


> 다음 분기 설계할 게임 분석

#  🕹️ Data Description

*   `Name` : 게임 이름
*   `Platform` : 게임이 지원되는 플랫폼의 이름
*   `Year` : 게임 출시 연도
*   `Genre` : 게임의 장르
*   `Publisher` : 게임을 제작한 회사
*   `NA_Sales` : 북미지역 출고량 (단위 :백만)
*   `EU_Sales` : 유럽지역에서의 출고량 (단위 :백만)
*   `JP_Sales` : 일본지역에서의 출고량 (단위 :백만)
*   `Other_Sales `: 기타지역 출고량 (단위 :백만)

#  데이터 탐색




## **1. Gathering Data**

In [1122]:
#라이브러리 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1123]:
#데이터셋 불러오기
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')
df.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


## **2. EDA**

In [1124]:
#불필요한 행 제거
df = df.iloc[:,1:]
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [1125]:
#결측치, 데이터타입 확인
#year, genre, publisher에 결측치 존재하고, 출고량과 year의 데이터타입 변경 필요.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.1+ MB


In [1126]:
#분석하기 쉽게 연도,이름순으로 정렬.
df.sort_values(by=['Year','Name'],ascending=False,inplace=True)

In [1127]:
#'Name'이 고유한지 확인하기 위해 이름의 중복치에 대해 이름과 연도를 기준으로 정렬
#같은 이름이어도 여러 플랫폼이 존재함.

df[df.duplicated(['Name'],keep=False)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
10107,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017.0,Role-Playing,Sega,0,0,0.03,0
15233,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017.0,Role-Playing,Sega,0,0,0.01,0
14837,ZombiU,PS4,2016.0,Action,Ubisoft,0,0.04,0.01,0.01
11424,Zero Escape: Zero Time Dilemma,PSV,2016.0,Adventure,Aksys Games,0.03,0,0.02,0.01
13073,Zero Escape: Zero Time Dilemma,3DS,2016.0,Adventure,Aksys Games,0.04,0,0.01,0.01
...,...,...,...,...,...,...,...,...,...
10534,Backbreaker,X360,NaN,Sports,Unknown,0.17,0,0,0.01
7150,Aquaman: Battle for Atlantis,XB,NaN,Action,Unknown,0.01,0,0,0
10896,All-Star Baseball 2005,PS2,NaN,Sports,Unknown,0.16,0.12,0,0.04
14119,All-Star Baseball 2005,XB,NaN,Sports,Unknown,0.11,0.03,0,0.01


In [1128]:
df[df.duplicated(['Name','Platform'],keep=False)]

# 같은 이름의 게임이어도 출시 년도가 다를 수 있음. (예를들면 재출시 하는 경우)
#그에 따라 'Year'은 대체가 불가능 하다고 판단됨.
#출고량에 단위가 'K'인것 발견

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1318,Need for Speed: Most Wanted,X360,2012.0,Racing,Electronic Arts,0.62,0.8,0.01,0.15
13808,Need for Speed: Most Wanted,PC,2012.0,Racing,Electronic Arts,0,0.06,0,0.02
7346,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,2.11,0.23,0,0.22
8717,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,0,0.01,0,0
15705,Sonic the Hedgehog,PS3,2006.0,Platform,Sega,0.41,0.07,0.04,0.66
2673,Need for Speed: Most Wanted,PC,2005.0,Racing,Electronic Arts,20K,0.23,0,0.05
12529,Need for Speed: Most Wanted,X360,2005.0,Racing,Electronic Arts,1,0.17,0.02,0.1
8055,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0,0,0.02,0
9184,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0,0,0.02,0
8651,Sonic the Hedgehog,PS3,NaN,Platform,NaN,0,0.48,0,0


In [1129]:
# Year 결측치들 제거
df.dropna(subset=['Year'],inplace=True)

In [1130]:
#year 의 데이터 타입 정수로 바꿔주기.
df['Year']=df['Year'].astype(int)

In [1131]:
#'Year'가 1900미만인 데이터 삭제.
df= df[df['Year'] > 1900]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16227 entries, 6906 to 5723
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         16227 non-null  object
 1   Platform     16227 non-null  object
 2   Year         16227 non-null  int64 
 3   Genre        16177 non-null  object
 4   Publisher    16191 non-null  object
 5   NA_Sales     16227 non-null  object
 6   EU_Sales     16227 non-null  object
 7   JP_Sales     16227 non-null  object
 8   Other_Sales  16227 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.2+ MB


In [1132]:
#Genre의 결측치와 같은 게임이름의 데이터를 가진 다른 열이 있는지 확인
missing_G = df[df['Genre'].isnull()]

In [1133]:
yes_G = df.copy()                       #원래 데이터를 복사해서 사용
yes_G = yes_G.dropna(subset=['Genre']) # 'Genre'가 nan값인 행들만 제거!

#기존데이터의 'Name'과 'Genre'결측치의 'Name'이 같은 데이터가 있는지 확인.
have_G = []

for data in yes_G['Name']:          
    if data in missing_G['Name']:
        have_G.append(data)
have_G                                  

[]

In [1134]:
#Genre결측치를 대체할 수 없으므로 전부 삭제
df.dropna(subset=['Genre'],inplace=True)

In [1135]:
#같은방법으로 'Publisher'확인 
missing_P = df[df['Publisher'].isnull()]
yes_P = df.copy()                       #원래 데이터를 복사해서 사용
yes_P = yes_P.dropna(subset=['Publisher']) # 'Publisher'가 nan값인 행들만 제거!

#기존데이터의 'Name'과 'Publisher'결측치의 'Name'이 같은 데이터가 있는지 확인.
have_P = []

for data in yes_P['Name']:           
    if data in missing_P['Name']:
        have_P.append(data)
have_P                                

[]

In [1136]:
df.isnull().sum() #결측치 갯수가 크지않기때문에 이 또한 삭제한다.

Name            0
Platform        0
Year            0
Genre           0
Publisher      36
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
dtype: int64

In [1137]:
#결측치 삭제
df.dropna(inplace=True)

In [1138]:
#'Name','Platform','Year' 이 같은 항목 삭제
df.drop_duplicates(['Name','Platform','Year'],keep=False,inplace=True)

In [1139]:
df.reset_index(drop=True,inplace=True)

In [1140]:
import re

def has_errors(inputString):
  return bool(re.search('[a-zA-Z]+', inputString))

error_condition = df.NA_Sales.apply(has_errors)|df.EU_Sales.apply(has_errors)|df.JP_Sales.apply(has_errors)|df.Other_Sales.apply(has_errors)
error= df[error_condition]
error

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Imagine: Makeup Artist,DS,2020,Simulation,Ubisoft,0.27,0K,0,0.02
37,Touhou Genso Rondo: Bullet Ballet,PS4,2016,Shooter,Nippon Ichi Software,0.01M,0,0,0
38,Total War: WARHAMMER,PC,2016,Strategy,Sega,0M,0.1,0,0.01
51,The Idolmaster: Platinum Stars,PS4,2016,Simulation,Namco Bandai Games,0M,0,0.09,0
84,Song of the Deep,PS4,2016,Action,Insomniac Games,0.05M,0,0,0.01
...,...,...,...,...,...,...,...,...,...
15993,Pro Yakyuu Family Stadium,NES,1986,Sports,Namco Bandai Games,0.15,0,2.05,0M
16010,Ice Climber,NES,1985,Platform,Nintendo,0.46M,0.1,0.92,0.02
16026,F1 Race,NES,1984,Racing,Nintendo,0,0,1.52,0K
16109,E.T.: The Extra Terrestrial,2600,1981,Action,Atari,1.84,110K,0,0.02


In [1141]:
#출고량에 'K'나 'M'등 문자로 이루어진 데이터 변환해줌.
#'M'은 원래 단위와 같으니 지워주고
# 'K'인 데이터는 *0.001 을 해준다.

columns= ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']

for i in columns:
  df[i] =df[i].replace({"K":"*0.001", "M":""}, regex=True).map(pd.eval)


# 데이터 분석